In [1]:
import colorsys
import os
import re
import pickle
from timeit import default_timer as timer

import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
import os
from keras.utils import multi_gpu_model, plot_model
# from tensorflow.python.keras.utils.vis_utils import plot_model

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

# 既存モデルの読み出し

In [2]:
model = tiny_yolo_body(Input(shape=(None, None, 3)), 3, 1)

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
model.load_weights('logs/000/trained_weights_final_part1.h5')

In [2]:
#model = load_model('logs/000/trained_weights_final.h5')

Instructions for updating:
Colocations handled automatically by placer.


NameError: name 'yolo_head' is not defined

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 1 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 1 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 1 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [7]:
len(model.layers)

44

In [32]:
partial_params = model.layers[22].get_weights()[0]

In [6]:
model.layers[22].name

'batch_normalization_6'

# 重みパラメータのゲット

In [4]:
weights_layer = {}
weights_layer["conv"] = {}
weights_layer["bn"] = {}
for i, lay in enumerate(model.layers):
    if re.search("conv", lay.name):
        weights_layer["conv"][lay.name] = {}
        weights_layer["conv"][lay.name]["weight"] = lay.get_weights()
        weights_layer["conv"][lay.name]["layer_id"] = i
    
    elif re.search("batch_normalization", lay.name):
        weights_layer["bn"][lay.name] = {}
        weights_layer["bn"][lay.name]["weight"] = lay.get_weights()
        weights_layer["bn"][lay.name]["layer_id"] = i

In [9]:
len(weights_layer["bn"])

11

In [6]:
weights_layer["conv"].keys()

dict_keys(['conv2d_1', 'conv2d_2', 'conv2d_3', 'conv2d_4', 'conv2d_5', 'conv2d_6', 'conv2d_7', 'conv2d_8', 'conv2d_11', 'conv2d_9', 'conv2d_12', 'conv2d_10', 'conv2d_13'])

In [7]:
weights_layer["conv"]

{'conv2d_1': {'layer_id': 1,
  'weight': [array([[[[ 1.89038128e-01, -2.20744506e-01,  1.29804134e-01,
              3.53077233e-01, -6.36183798e-01,  1.46289945e+00,
              6.28831208e-01, -1.35358885e-01, -7.15234101e-01,
             -1.53451860e+00,  1.32872844e+00,  1.35110331e+00,
             -4.68614429e-01,  3.21613669e-01, -1.64543045e+00,
              4.48554680e-02],
            [ 1.88686728e-01,  7.04186335e-02,  9.18381140e-02,
             -3.17954838e-01, -6.93003178e-01, -7.24363208e-01,
              3.57972056e-01,  1.10006846e-01, -1.30685776e-01,
             -1.31460857e+00,  1.11659074e+00,  1.47978115e+00,
              3.76533359e-01,  4.34941947e-01, -1.27904367e+00,
             -3.70395839e-01],
            [-2.87504435e-01,  3.97850156e-01, -3.17093134e-01,
             -9.85752791e-03, -3.72571945e-01, -9.18410718e-01,
              2.37075180e-01,  6.51706755e-02,  2.09620014e-01,
             -8.01033437e-01,  6.74377084e-01,  8.96017492e-01,
   

In [14]:
model.layers[43].set_weights(weights_layer["conv"]["conv2d_26"]["weight"])

In [15]:
weights_layer["conv"]["conv2d_20"]["weight"][0].shape

(3, 3, 512, 1024)

In [8]:
weights_layer["conv"].keys()

dict_keys(['conv2d_1', 'conv2d_2', 'conv2d_3', 'conv2d_4', 'conv2d_5', 'conv2d_6', 'conv2d_7', 'conv2d_8', 'conv2d_11', 'conv2d_9', 'conv2d_12', 'conv2d_10', 'conv2d_13'])

In [5]:
layer_list = weights_layer.keys()
layer_num = len(weights_layer.keys())

reduced_weights = {}
reduced_weights["conv"] = {}
reduced_weights["bn"] = {}

for layer_name in list(weights_layer["conv"].keys()):
    conv_num = int(layer_name.split("_")[1])
    print(conv_num,layer_name)
    #if conv_num%2 != 0:
    reduced_weights["conv"][layer_name] = weights_layer["conv"][layer_name]
    #with open("./{}.pickle".format(layer_name),mode="wb") as f:
    #     pickle.dump(weights_layer["bn"][layer_name],f)
for i, layer_name in enumerate(list(weights_layer["bn"].keys())):
    reduced_weights["bn"][layer_name] = weights_layer["bn"][layer_name]

1 conv2d_1
2 conv2d_2
3 conv2d_3
4 conv2d_4
5 conv2d_5
6 conv2d_6
7 conv2d_7
8 conv2d_8
11 conv2d_11
9 conv2d_9
12 conv2d_12
10 conv2d_10
13 conv2d_13


In [6]:
reduced_weights["conv"].keys()

dict_keys(['conv2d_1', 'conv2d_2', 'conv2d_3', 'conv2d_4', 'conv2d_5', 'conv2d_6', 'conv2d_7', 'conv2d_8', 'conv2d_11', 'conv2d_9', 'conv2d_12', 'conv2d_10', 'conv2d_13'])

In [7]:
with open("./weights_part1.pickle",mode="wb") as f:
    pickle.dump(reduced_weights,f)

In [10]:
plot_model(model,"tiny_model.png")

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

## ↓別コードに移動 

# 重みパラメータのセット

In [2]:
num_anchors = 3
num_classes = 1
first_model = tiny_yolo_body(Input(shape=(None, None, 3)), 3, 1)

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
first_model.layers[1].get_weights()

[array([[[[-0.11105562,  0.04261331, -0.17474808,  0.1061863 ,
           -0.04748504,  0.10141337, -0.04180804, -0.11033213,
           -0.18034093,  0.02832322,  0.062684  ,  0.16798198,
           -0.10719227, -0.16038877,  0.08861971, -0.01524158],
          [ 0.02140178,  0.02948406, -0.15941183, -0.00215234,
            0.16339898, -0.05143905,  0.08222765, -0.05903296,
           -0.11089694, -0.06241769,  0.17199108,  0.00452124,
            0.18448737,  0.05689742,  0.14798096, -0.17877266],
          [ 0.01919581,  0.1579704 , -0.07542632, -0.08500957,
           -0.07662632,  0.08856332,  0.00079007,  0.09059116,
            0.04410584,  0.14293501, -0.17127462, -0.13688347,
           -0.0330109 , -0.13705045, -0.12324866,  0.06122629]],
 
         [[-0.03815138,  0.04366711, -0.18192828,  0.13645986,
            0.181027  ,  0.1296637 , -0.00062068, -0.14260288,
           -0.03873838,  0.17802331, -0.06858759,  0.11938006,
           -0.06459098, -0.17646495, -0.14610381,

In [4]:
first_model.layers[5].get_weights()

[array([[[[-0.11542212,  0.04912055, -0.00199602, ..., -0.103764  ,
            0.07737061, -0.0463955 ],
          [-0.0424398 ,  0.01835447, -0.00609856, ..., -0.10724977,
           -0.10177694,  0.0275814 ],
          [ 0.03295859,  0.00096342, -0.10373337, ...,  0.08218484,
           -0.09343535,  0.02939231],
          ...,
          [-0.03219606, -0.11089209, -0.05326432, ..., -0.01885968,
           -0.08240911, -0.10399999],
          [ 0.07278629, -0.02237646,  0.08985213, ..., -0.0346932 ,
           -0.02082473,  0.11493299],
          [-0.100256  ,  0.06299891,  0.08633215, ..., -0.10734063,
           -0.03675935, -0.00722594]],
 
         [[-0.09332939, -0.05912037,  0.02857179, ...,  0.04076651,
           -0.0248592 , -0.10212816],
          [ 0.05457482, -0.01310328, -0.03177355, ..., -0.00823052,
           -0.05183374,  0.10849344],
          [-0.02104308, -0.04122521,  0.05440711, ..., -0.06835045,
           -0.00731242, -0.05745456],
          ...,
          [-0

In [3]:
first_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 1 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 1 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 1 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [14]:
first_layer = {}
first_layer["conv"] = {}
first_layer["bn"] = {}
for i, lay in enumerate(first_model.layers):
    if re.search("conv", lay.name):
        first_layer["conv"][lay.name] = {}
        first_layer["conv"][lay.name]["weight"] = lay.get_weights()
        first_layer["conv"][lay.name]["layer_id"] = i
        #weights_layer.append(ln.name)
    
    elif re.search("batch_normalization", lay.name):
        first_layer["bn"][lay.name] = {}
        first_layer["bn"][lay.name]["weight"] = lay.get_weights()
        first_layer["bn"][lay.name]["layer_id"] = i
        
first_layer["conv"]["conv2d_6"]

{'layer_id': 21,
 'weight': [array([[[[ 0.01135941, -0.02073381, -0.02327976, ...,  0.02349702,
             0.01586056,  0.01774974],
           [-0.00142265, -0.00889246, -0.01692614, ..., -0.02693716,
            -0.01882637, -0.02325586],
           [ 0.02656271,  0.00410049, -0.02696598, ...,  0.00148378,
            -0.00340509, -0.01830549],
           ...,
           [-0.02193097,  0.00620647,  0.0091499 , ..., -0.01855088,
            -0.00045238, -0.02175311],
           [ 0.0153889 , -0.02883781, -0.01023538, ...,  0.01365518,
            -0.0185838 , -0.01443596],
           [ 0.00549707, -0.02189761, -0.00607021, ..., -0.02510266,
            -0.00469476,  0.01835822]],
  
          [[ 0.00876666,  0.01996232,  0.01598621, ..., -0.00185842,
             0.01236595,  0.00866449],
           [-0.01651628, -0.02032185,  0.0157322 , ..., -0.02610878,
            -0.00477733, -0.02282943],
           [-0.00380545,  0.02057193, -0.01459773, ..., -0.02025447,
            -0.02167

In [5]:
with open("weights.pickle",mode="rb") as f:
    data = pickle.load(f)

In [7]:
data.keys()

dict_keys(['conv', 'bn'])

In [8]:
data["conv"]["conv2d_3"]["layer_id"]

9

In [5]:
data["conv"].keys()

dict_keys(['conv2d_1', 'conv2d_3', 'conv2d_5', 'conv2d_7', 'conv2d_11', 'conv2d_9', 'conv2d_13'])

In [6]:
for conv_layer in data["conv"].keys():
    layer_id = data["conv"][conv_layer]["layer_id"]
    print(layer_id,conv_layer)
    first_model.layers[layer_id].set_weights(data["conv"][conv_layer]["weight"])

1 conv2d_1
9 conv2d_3
17 conv2d_5
25 conv2d_7
31 conv2d_11
36 conv2d_9
43 conv2d_13


In [7]:
first_model.layers[1].get_weights()

[array([[[[ 1.83500513e-01, -2.19381079e-01,  1.31666571e-01,
            3.55932206e-01, -6.32375360e-01,  1.46276987e+00,
            6.33728981e-01, -1.43535525e-01, -7.12047637e-01,
           -1.53899145e+00,  1.32394505e+00,  1.34851170e+00,
           -4.60721940e-01,  3.25104445e-01, -1.64741528e+00,
            4.12780643e-02],
          [ 1.83558702e-01,  7.04849809e-02,  9.26005021e-02,
           -3.14925194e-01, -6.89887464e-01, -7.22762108e-01,
            3.61856580e-01,  1.03791811e-01, -1.25472248e-01,
           -1.31889427e+00,  1.11143613e+00,  1.47715843e+00,
            3.84297520e-01,  4.38244969e-01, -1.28101492e+00,
           -3.73884082e-01],
          [-2.91188657e-01,  3.98754209e-01, -3.15291524e-01,
           -8.18840042e-03, -3.70252937e-01, -9.17159438e-01,
            2.41084099e-01,  6.51152208e-02,  2.14512870e-01,
           -8.05864811e-01,  6.70215189e-01,  8.92325997e-01,
            2.88612664e-01,  4.11996365e-01, -9.66226816e-01,
            

In [8]:
first_model.layers[5].get_weights()

[array([[[[-0.11542212,  0.04912055, -0.00199602, ..., -0.103764  ,
            0.07737061, -0.0463955 ],
          [-0.0424398 ,  0.01835447, -0.00609856, ..., -0.10724977,
           -0.10177694,  0.0275814 ],
          [ 0.03295859,  0.00096342, -0.10373337, ...,  0.08218484,
           -0.09343535,  0.02939231],
          ...,
          [-0.03219606, -0.11089209, -0.05326432, ..., -0.01885968,
           -0.08240911, -0.10399999],
          [ 0.07278629, -0.02237646,  0.08985213, ..., -0.0346932 ,
           -0.02082473,  0.11493299],
          [-0.100256  ,  0.06299891,  0.08633215, ..., -0.10734063,
           -0.03675935, -0.00722594]],
 
         [[-0.09332939, -0.05912037,  0.02857179, ...,  0.04076651,
           -0.0248592 , -0.10212816],
          [ 0.05457482, -0.01310328, -0.03177355, ..., -0.00823052,
           -0.05183374,  0.10849344],
          [-0.02104308, -0.04122521,  0.05440711, ..., -0.06835045,
           -0.00731242, -0.05745456],
          ...,
          [-0

In [16]:
first_model.layers[1].set_weights(data["conv"]["conv2d_1"]["weight"])

In [ ]:
data["conv"]["conv2d_1"]

In [19]:
data["conv"].keys()

dict_keys(['conv2d_1', 'conv2d_3', 'conv2d_5', 'conv2d_11', 'conv2d_9', 'conv2d_13'])

In [21]:
first_model.layers[21].get_weights()

[array([[[[-2.88995318e-02, -1.54270884e-02, -2.11981088e-02, ...,
           -5.00999242e-02,  3.53235379e-02, -1.19346110e-02],
          [ 1.88221950e-02, -4.15889099e-02,  4.64024507e-02, ...,
            2.60163248e-02, -2.87014320e-02, -2.54389644e-02],
          [ 6.68064952e-02, -2.20363792e-02,  4.05505747e-02, ...,
           -3.08312997e-02, -2.44866610e-02,  5.38429618e-03],
          ...,
          [ 2.60877348e-02, -2.25735316e-03, -1.41791422e-02, ...,
            9.75113455e-03,  7.32165873e-02,  5.86484708e-02],
          [-5.07447161e-02, -1.64534505e-02, -1.77660361e-02, ...,
            1.44261699e-02,  9.43748876e-02, -3.46791856e-02],
          [ 5.58648854e-02, -4.15572058e-03,  5.91850206e-02, ...,
           -6.50383383e-02, -1.78503722e-01, -2.17641480e-02]],
 
         [[ 3.66641325e-03,  3.19722220e-02, -5.14471531e-02, ...,
           -8.98601301e-03,  4.00068946e-02,  5.15443534e-02],
          [ 4.16580439e-02, -3.80963236e-02,  2.99213566e-02, ...,
     

In [20]:
!pip3 install graphviz

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
  Using cached graphviz-0.14.2-py2.py3-none-any.whl (18 kB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [23]:
!pip3 install pydot-ng

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
  Using cached pydot_ng-2.0.0-py2.py3-none-any.whl (20 kB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
!pip install pydotplus

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 278 kB 3.0 MB/s eta 0:00:01
  Created wheel for pydotplus: filename=pydotplus-2.0.2-py2-none-any.whl size=25392 sha256=26d41da9d2b9fafa06d953b395f0f569ce6ec05c69ebab520788c9cca48ae5af
  Stored in directory: /home/shusaku/.cache/pip/wheels/4f/ce/27/eeeaa899ac9a92551988a435261b88f05e84b14a7a92e8393d
Successfully built pydotplus
You should consider upgrading via t